# Using the cache

First make sure that the dependencies are installed:

```bash
pip install -r requirements-dev.txt
```

Then run redis using docker compose:

```bash
docker compose -f docker-compose.redis.yml up -d
```

Then run the notebook

In [1]:
from redis_block_fsspec_cache import RedisCachingFileSystem

fs = RedisCachingFileSystem(
    redis_host="localhost",
    redis_port=6380,
    expiry_time=60,
    target_protocol="s3",
    target_options={
        'anon': True,
    },
)

Let try to access a single chunk from a remote GRIB file we know about in an s3 bucket:

```json
"gust\/0.0.0": [
    "s3:\/\/noaa-hrrr-bdp-pds\/hrrr.20230927\/conus\/hrrr.t00z.wrfsubhf00.grib2",
    2884409,
    1133540
],
```

In [2]:
%%time

uncached_chunk = fs.read_block("s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf00.grib2", offset=2884409, length=1133540)

HITTT
CPU times: user 161 ms, sys: 114 ms, total: 275 ms
Wall time: 3.15 s


In [3]:
%%time

cached_chunk = fs.read_block("s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf00.grib2", offset=2884409, length=1133540)

HITTT
CPU times: user 17.9 ms, sys: 24.3 ms, total: 42.2 ms
Wall time: 230 ms


In [7]:
print(uncached_chunk == cached_chunk)

True
